The spelled-out intro to neural networks and backpropagation: building micrograd
https://www.youtube.com/watch?v=VMj-3S1tku0

In [1]:
print("hello, world!")

hello, world!


In [3]:
import math
# import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'